### Set-up

In [104]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!cd /content/drive/MyDrive/Mantises!/

Mounted at /content/drive


In [105]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import ast
import numpy as np
import os
from sklearn.tree import DecisionTreeClassifier, plot_tree
import matplotlib.pyplot as plt

### Join county and biodiversity data



In [106]:
biodiversity_data = pd.read_csv(f'/content/drive/My Drive/Mantises!/data/mantodea_ca_gbif_simple.csv', sep='\t')

status = {
    'Mantis religiosa': 'introduced',
    'Stagmomantis californica': 'native',
    'Litaneutria pacifica': 'native',
    'Litaneutria skinneri': 'native',
    'Stagmomantis limbata': 'native',
    'Iris oratoria': 'introduced',
    'Miomantis caffra': 'introduced',
    'Litaneutria ocularis': 'native',
    'Tenodera sinensis': 'introduced',
    'Litaneutria chaparrali': 'native',
    'Hierodula patellifera': 'introduced',
    'Litaneutria minor': 'native',
    'Yersiniops newboldi': 'native',
    'Thesprotia graminis': 'native'
}

# Add status column
biodiversity_data['status'] = biodiversity_data['species'].map(status)

unique_species = biodiversity_data['species'].unique()

biodiversity_data.head()

,gbifID,datasetKey,occurrenceID,kingdom,phylum,class,order,family,genus,species,...,dateIdentified,license,rightsHolder,recordedBy,typeStatus,establishmentMeans,lastInterpreted,mediaType,issue,status
0,923926829,50c9509d-22c7-4a22-a47d-8c48425ef4a7,http://www.inaturalist.org/observations/766957,Animalia,Arthropoda,Insecta,Mantodea,Mantidae,Mantis,Mantis religiosa,...,2014-07-03T20:54:23,CC_BY_NC_4_0,Callahan Charleton,Callahan Charleton,NaN,NaN,2023-09-28T11:04:15.229Z,StillImage,CONTINENT_DERIVED_FROM_COORDINATES;TAXON_MATCH...,introduced
1,923921190,50c9509d-22c7-4a22-a47d-8c48425ef4a7,http://www.inaturalist.org/observations/750035,Animalia,Arthropoda,Insecta,Mantodea,Mantidae,Mantis,Mantis religiosa,...,2014-06-23T01:27:47,CC0_1_0,Tony Iwane,Tony Iwane,NaN,NaN,2023-09-28T12:35:45.908Z,StillImage,COORDINATE_ROUNDED;CONTINENT_DERIVED_FROM_COOR...,introduced
2,923917961,50c9509d-22c7-4a22-a47d-8c48425ef4a7,http://www.inaturalist.org/observations/741586,Animalia,Arthropoda,Insecta,Mantodea,Mantidae,Mantis,Mantis religiosa,...,2014-06-18T17:43:13,CC_BY_NC_4_0,Todd Plummer,Todd Plummer,NaN,NaN,2023-09-28T11:04:17.149Z,StillImage,CONTINENT_DERIVED_FROM_COORDINATES;TAXON_MATCH...,introduced
3,899975780,50c9509d-22c7-4a22-a47d-8c48425ef4a7,http://www.inaturalist.org/observations/645036,Animalia,Arthropoda,Insecta,Mantodea,Mantidae,Mantis,Mantis religiosa,...,2014-04-27T06:25:51,CC_BY_NC_4_0,Eric Jacob,Eric Jacob,NaN,NaN,2023-09-28T12:34:59.254Z,StillImage,CONTINENT_DERIVED_FROM_COORDINATES;TAXON_MATCH...,introduced
4,891755288,50c9509d-22c7-4a22-a47d-8c48425ef4a7,http://www.inaturalist.org/observations/439613,Animalia,Arthropoda,Insecta,Mantodea,Mantidae,Mantis,Mantis religiosa,...,2013-10-25T04:13:21,CC_BY_NC_4_0,Paul G. Johnson,Paul G. Johnson,NaN,NaN,2023-09-28T11:04:05.380Z,StillImage,CONTINENT_DERIVED_FROM_COORDINATES;TAXON_MATCH...,introduced


In [107]:
# Load the shapefile with US counties
counties_shapefile = '/content/drive/My Drive/Mantises!/data/gadm41_USA_shp/gadm41_USA_2.shp'
counties = gpd.read_file(counties_shapefile)

# Convert the biodiversity data into a GeoDataFrame
geometry = [Point(xy) for xy in zip(biodiversity_data['decimalLongitude'], biodiversity_data['decimalLatitude'])]
biodiversity_gdf = gpd.GeoDataFrame(biodiversity_data, geometry=geometry)
biodiversity_gdf.set_crs(counties.crs, inplace=True)

# Spatial join to find which county each point falls into
joined = gpd.sjoin(biodiversity_gdf, counties, how='inner', predicate='within')

# Aggregate data to count occurrences per species, status, and county
aggregated_data1 = joined.groupby(['species', 'status', 'NAME_2']).size().reset_index(name='count')
aggregated_data1 = aggregated_data1.rename(columns={'NAME_2': 'County'})

# Export to CSV
output_file = '/content/drive/My Drive/Mantises!/output/mantises_per_county.csv'
aggregated_data1.to_csv(output_file, index=False)

aggregated_data1.head()

,species,status,County,count
0,Hierodula patellifera,introduced,Alameda,1
1,Iris oratoria,introduced,Alameda,1
2,Iris oratoria,introduced,Butte,16
3,Iris oratoria,introduced,Contra Costa,11
4,Iris oratoria,introduced,Fresno,11


### Join table with climatic data

In [108]:
# Load the climate data
climate_data_path = '/content/drive/My Drive/Mantises!/output/climatic_data/california_climate_dummy_data.csv'
climate_data = pd.read_csv(climate_data_path)
climate_data = climate_data.drop(columns=['State'])

# Merge the climate data with the aggregated data using 'County' as the key
aggregated_data2 = pd.merge(aggregated_data1, climate_data, on='County', how='left')

aggregated_data2.head()


,species,status,County,count,Year,Month,tmax,tmin,prcp_monttl
0,Hierodula patellifera,introduced,Alameda,1,1981,11,31.22,NaN,NaN
1,Hierodula patellifera,introduced,Alameda,1,1981,10,23.49,NaN,NaN
2,Hierodula patellifera,introduced,Alameda,1,1981,2,NaN,NaN,19.09
3,Hierodula patellifera,introduced,Alameda,1,1981,5,NaN,14.26,NaN
4,Hierodula patellifera,introduced,Alameda,1,1981,3,NaN,15.42,NaN


### Join with ecoregion data

In [109]:
# Load the ecoregion data
ecoregion_data_path = '/content/drive/My Drive/Mantises!/output/ecoregions_per_county.csv'
ecoregion_data = pd.read_csv(ecoregion_data_path)

# Merge the ecoregion data with the merged data using 'County' as the key
aggregated_data3 = pd.merge(aggregated_data2, ecoregion_data, on='County', how='left')

aggregated_data3.head()


,species,status,County,count,Year,Month,tmax,tmin,prcp_monttl,Unique_Ecoregions
0,Hierodula patellifera,introduced,Alameda,1,1981,11,31.22,NaN,NaN,{'Central California Foothills and Coastal Mou...
1,Hierodula patellifera,introduced,Alameda,1,1981,10,23.49,NaN,NaN,{'Central California Foothills and Coastal Mou...
2,Hierodula patellifera,introduced,Alameda,1,1981,2,NaN,NaN,19.09,{'Central California Foothills and Coastal Mou...
3,Hierodula patellifera,introduced,Alameda,1,1981,5,NaN,14.26,NaN,{'Central California Foothills and Coastal Mou...
4,Hierodula patellifera,introduced,Alameda,1,1981,3,NaN,15.42,NaN,{'Central California Foothills and Coastal Mou...


In [110]:
# Convert strings to list of ecoregions
unique_ecoregions = set()
for ecoregion_set in data['Unique_Ecoregions']:
    # Safely evaluate the string as a Python literal (set)
    # If the column contains actual sets, you can skip the ast.literal_eval part
    ecoregion_set = ast.literal_eval(ecoregion_set)
    unique_ecoregions.update(ecoregion_set)

print(unique_ecoregions)

{'Sierra Nevada', 'Southern California/Northern Baja Coast', 'Sonoran Basin and Range', 'Central California Foothills and Coastal Mountains', 'Central Basin and Range', 'Southern California Mountains', 'Mojave Basin and Range', 'Eastern Cascades Slopes and Foothills', 'Northern Basin and Range', 'Central California Valley', 'Cascades', 'Klamath Mountains/California High North Coast Range', 'Coast Range'}


### Join with human population data

In [111]:
# Load the human population
human_population_data_path = '/content/drive/My Drive/Mantises!/output/human_population_per_county.csv'
human_population_data = pd.read_csv(human_population_data_path)
human_population_data = human_population_data.rename(columns={'NAME_2': 'County'})
human_population_data = human_population_data.drop(columns=['geometry'])

# Merge the human population data with the existing data using 'County' as the key
aggregated_data4 = pd.merge(aggregated_data3, human_population_data, on='County', how='left')

# Export to CSV
output_file = '/content/drive/My Drive/Mantises!/output/aggregated_data_per_county.csv'
aggregated_data4.to_csv(output_file, index=False)

aggregated_data4.head()


,species,status,County,count,Year,Month,tmax,tmin,prcp_monttl,Unique_Ecoregions,population_total
0,Hierodula patellifera,introduced,Alameda,1,1981,11,31.22,NaN,NaN,{'Central California Foothills and Coastal Mou...,1661584
1,Hierodula patellifera,introduced,Alameda,1,1981,10,23.49,NaN,NaN,{'Central California Foothills and Coastal Mou...,1661584
2,Hierodula patellifera,introduced,Alameda,1,1981,2,NaN,NaN,19.09,{'Central California Foothills and Coastal Mou...,1661584
3,Hierodula patellifera,introduced,Alameda,1,1981,5,NaN,14.26,NaN,{'Central California Foothills and Coastal Mou...,1661584
4,Hierodula patellifera,introduced,Alameda,1,1981,3,NaN,15.42,NaN,{'Central California Foothills and Coastal Mou...,1661584


### Pre-process data

In [112]:
# Aggregate climatic data and population total for each county
county_climate_pop = aggregated_data4.groupby('County').agg({'tmax': 'mean', 'tmin': 'mean', 'prcp_monttl': 'mean', 'population_total': 'first'}).reset_index()

# Prepare a DataFrame with all combinations of species and counties
unique_species = aggregated_data4['species'].unique()
unique_counties = aggregated_data4['County'].unique()
all_combinations = pd.MultiIndex.from_product([unique_species, unique_counties], names=['species', 'County']).to_frame(index=False)

# Merge with aggregated climate data and population data
merged_data = all_combinations.merge(county_climate_pop, on='County', how='left')

# Add binary columns for each ecoregion
unique_ecoregions = {ecoregion for ecoregion_list in aggregated_data4['Unique_Ecoregions'].apply(eval) for ecoregion in ecoregion_list}
for ecoregion in unique_ecoregions:
    merged_data['Eco_' + ecoregion.replace(' ', '_')] = merged_data['County'].map(aggregated_data4.groupby('County')['Unique_Ecoregions'].apply(lambda x: ecoregion in ' '.join(x)))

# Map the original species presence data
species_presence_map = aggregated_data4.drop_duplicates(subset=['species', 'County']).set_index(['species', 'County'])['count']
merged_data['species_presence'] = merged_data.set_index(['species', 'County']).index.map(species_presence_map).fillna(0).astype(int)
merged_data['species_presence'] = merged_data['species_presence'].apply(lambda x: 1 if x > 0 else 0)

merged_data.head()


,species,County,tmax,tmin,prcp_monttl,population_total,Eco_Sierra_Nevada,Eco_Southern_California/Northern_Baja_Coast,Eco_Sonoran_Basin_and_Range,Eco_Central_California_Foothills_and_Coastal_Mountains,Eco_Central_Basin_and_Range,Eco_Southern_California_Mountains,Eco_Mojave_Basin_and_Range,Eco_Eastern_Cascades_Slopes_and_Foothills,Eco_Northern_Basin_and_Range,Eco_Central_California_Valley,Eco_Cascades,Eco_Klamath_Mountains/California_High_North_Coast_Range,Eco_Coast_Range,species_presence
0,Hierodula patellifera,Alameda,26.572426,10.113893,9.689462,1661584,False,False,False,True,False,False,False,False,False,True,False,False,False,1
1,Hierodula patellifera,Butte,27.455577,9.708830,10.432373,223344,True,False,False,True,False,False,False,False,False,True,True,False,False,0
2,Hierodula patellifera,Contra Costa,27.246092,9.513614,10.030976,1147788,False,False,False,True,False,False,False,False,False,True,False,False,False,0
3,Hierodula patellifera,Fresno,27.235495,9.875361,9.115385,990204,True,False,False,True,False,False,False,False,False,True,False,False,False,0
4,Hierodula patellifera,Glenn,28.241419,10.096343,9.792126,28060,False,False,False,True,False,False,False,False,False,True,False,True,True,0


### Decision trees per species

In [97]:

feature_columns = ['tmax', 'tmin', 'prcp_monttl', 'population_total'] + \
    [col for col in merged_data.columns if col.startswith('Eco_')]

# Output directory for decision tree visualizations
output_dir = '/content/drive/My Drive/Mantises!/output/decision_trees'
os.makedirs(output_dir, exist_ok=True)

# Iterate over each species to build a model and visualize the decision tree
for species in unique_species:
    # Filter data for the current species
    species_data = merged_data[merged_data['species'] == species]
    X = species_data[feature_columns]
    y = species_data['species_presence']

    # Initialize and train the decision tree classifier
    model = DecisionTreeClassifier(criterion='entropy', random_state=42)
    model.fit(X, y)

    # Visualize the decision tree
    plt.figure(figsize=(20, 10))
    plot_tree(model, feature_names=feature_columns, class_names=['Absent', 'Present'], filled=True)
    plt.title(f"Decision Tree for {species}")

    # Save the figure
    plt.savefig(os.path.join(output_dir, f"decision_tree_{species}.png"))
    plt.close()

    # Extract and print feature importances
    feature_importances = model.feature_importances_
    importance_df = pd.DataFrame(feature_importances, index=feature_columns, columns=['Importance']).sort_values(by='Importance', ascending=False)
    print(f"Feature Importances for {species}:\n", importance_df)
    print("\n")



Feature Importances for Hierodula patellifera:
                                                     Importance
population_total                                           1.0
tmax                                                       0.0
Eco_Southern_California_Mountains                          0.0
Eco_Klamath_Mountains/California_High_North_Coa...         0.0
Eco_Cascades                                               0.0
Eco_Central_California_Valley                              0.0
Eco_Northern_Basin_and_Range                               0.0
Eco_Eastern_Cascades_Slopes_and_Foothills                  0.0
Eco_Mojave_Basin_and_Range                                 0.0
Eco_Central_Basin_and_Range                                0.0
tmin                                                       0.0
Eco_Central_California_Foothills_and_Coastal_Mo...         0.0
Eco_Sonoran_Basin_and_Range                                0.0
Eco_Southern_California/Northern_Baja_Coast                0.0
Eco_Sie

### Decision trees for introduced/native

In [98]:

# Prepare the data
aggregated_data_status = aggregated_data4.copy()
aggregated_data_status['status_binary'] = aggregated_data_status['status'].apply(lambda x: 1 if x == 'introduced' else 0)

# Output directory for decision tree visualizations
output_dir = '/content/drive/My Drive/Mantises!/output/decision_trees'
os.makedirs(output_dir, exist_ok=True)

# Features to use for the model
feature_columns = ['tmax', 'tmin', 'prcp_monttl', 'population_total'] + \
    [col for col in aggregated_data_status.columns if col.startswith('Eco_')]

# Initialize and train the decision tree classifier
X = aggregated_data_status[feature_columns]
y = aggregated_data_status['status_binary']

model = DecisionTreeClassifier(criterion='entropy', random_state=42)
model.fit(X, y)

# Visualize the decision tree
plt.figure(figsize=(20, 10))
plot_tree(model, feature_names=feature_columns, class_names=['Native', 'Introduced'], filled=True)
plt.title("Decision Tree for Introduced vs. Native Status")

# Save the figure
plt.savefig(os.path.join(output_dir, "decision_tree_status.png"))
plt.close()

# Extract and print feature importances
feature_importances = model.feature_importances_
importance_df = pd.DataFrame(feature_importances, index=feature_columns, columns=['Importance']).sort_values(by='Importance', ascending=False)
print("Feature Importances for Introduced vs. Native Status:\n", importance_df)


ValueError: ignored

In [100]:
# Check for NaN values in the feature columns
nan_check = aggregated_data_status[feature_columns].isna().sum()
print(nan_check)


tmax                72912
tmin                73255
prcp_monttl         73577
population_total        0
dtype: int64


In [113]:
# Check for NaN values in the feature columns
nan_check = aggregated_data4[feature_columns].isna().sum()
print(nan_check)


tmax                72912
tmin                73255
prcp_monttl         73577
population_total        0
dtype: int64


In [114]:
for species in unique_species:
    species_data = merged_data[merged_data['species'] == species]
    print(f"{species} - NaN Counts:")
    print(species_data[feature_columns].isna().sum())
    print("\n")


Hierodula patellifera - NaN Counts:
tmax                0
tmin                0
prcp_monttl         0
population_total    0
dtype: int64


Iris oratoria - NaN Counts:
tmax                0
tmin                0
prcp_monttl         0
population_total    0
dtype: int64


Litaneutria chaparrali - NaN Counts:
tmax                0
tmin                0
prcp_monttl         0
population_total    0
dtype: int64


Litaneutria minor - NaN Counts:
tmax                0
tmin                0
prcp_monttl         0
population_total    0
dtype: int64


Litaneutria ocularis - NaN Counts:
tmax                0
tmin                0
prcp_monttl         0
population_total    0
dtype: int64


Litaneutria pacifica - NaN Counts:
tmax                0
tmin                0
prcp_monttl         0
population_total    0
dtype: int64


Litaneutria skinneri - NaN Counts:
tmax                0
tmin                0
prcp_monttl         0
population_total    0
dtype: int64


Mantis religiosa - NaN Counts:
tmax      